In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Training AI/IMDB_Top250Engmovies2_OMDB_Detailed.csv')
df

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,246,The Lost Weekend,1945,NOT RATED,01 Jan 1946,101 min,"Drama, Film-Noir",Billy Wilder,"Charles R. Jackson (from the novel by), Charle...","Ray Milland, Jane Wyman, Phillip Terry, Howard...",The desperate life of a chronic alcoholic is f...,English,USA,Won 4 Oscars. Another 12 wins & 3 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.0/10,NaN,8.0,"26,834",tt0037884,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.co

In [ ]:
df = df[['Title', 'Genre', 'Director', 'Actors','Plot']]
df

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...
...,...,...,...,...,...
245,The Lost Weekend,"Drama, Film-Noir",Billy Wilder,"Ray Milland, Jane Wyman, Phillip Terry, Howard...",The desperate life of a chronic alcoholic is f...
246,Short Term 12,Drama,Destin Daniel Cretton,"Brie Larson, John Gallagher Jr., Stephanie Bea...",A 20-something supervising staff member of a r...
247,His Girl Friday,"Comedy, Drama, Romance",Howard Hawks,"Cary Grant, Rosalind Russell, Ralph Bellamy, G...",A newspaper editor uses every trick in the boo...
248,The Straight Story,"Biography, Drama",David Lynch,"Sissy Spacek, Jane Galloway Heitz, Joseph A. C...",An old man makes a long journey by lawn-mover ...


In [ ]:
df.shape

(250, 5)

In [ ]:
df['Actors']

0      Tim Robbins, Morgan Freeman, Bob Gunton, Willi...
1      Marlon Brando, Al Pacino, James Caan, Richard ...
2      Al Pacino, Robert Duvall, Diane Keaton, Robert...
3      Christian Bale, Heath Ledger, Aaron Eckhart, M...
4      Martin Balsam, John Fiedler, Lee J. Cobb, E.G....
                             ...                        
245    Ray Milland, Jane Wyman, Phillip Terry, Howard...
246    Brie Larson, John Gallagher Jr., Stephanie Bea...
247    Cary Grant, Rosalind Russell, Ralph Bellamy, G...
248    Sissy Spacek, Jane Galloway Heitz, Joseph A. C...
249    Dev Patel, Saurabh Shukla, Anil Kapoor, Raj Zu...
Name: Actors, Length: 250, dtype: object

In [ ]:
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('wordnet')
# Init the Wordnet Lemmatizer
lem = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def lem_word(text):
    tokens = [lem.lemmatize(word, pos = 'n') for word in text]
    tokens = [lem.lemmatize(word, pos = 'v') for word in tokens]
    tokens = [lem.lemmatize(word, pos = 'a') for word in tokens]

    return tokens


In [ ]:
df['splited_Actors'] = df['Actors'].apply(lambda x: x.split(', '))

df['splited_Genre'] = df['Genre'].apply(lambda x: x.lower().split(','))

df['lower_plot'] = df['Plot'].apply(lambda x: x.lower())

df['lower_Director'] = df['Director'].apply(lambda x: x.lower().replace(' ','').split(','))

df['cleaned_actors'] = df['Actors'].apply(lambda x: x.lower().replace(' ','').split(','))

df['combine'] = df['splited_Genre'] + df['lower_Director'] + df['cleaned_actors'] + df['lower_plot'].apply(lambda x: x.split())

df['lemmatize_text'] = df['combine'].apply(lem_word)

df['final_words'] = df['lemmatize_text'].apply(lambda x: ' '.join(x))

df_final = df[['Title', 'final_words']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [ ]:
df_final['count_words'] = df['final_words'].apply(lambda x: len(x.split()))
df_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Title,final_words,count_words
0,The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...,26
1,The Godfather,crime drama francisfordcoppola marlonbrando a...,25
2,The Godfather: Part II,crime drama francisfordcoppola alpacino rober...,35
3,The Dark Knight,action crime drama christophernolan christia...,49
4,12 Angry Men,crime drama sidneylumet martinbalsam johnfied...,25
...,...,...,...
245,The Lost Weekend,drama film-noir billywilder raymilland janewy...,21
246,Short Term 12,drama destindanielcretton brielarson johngalla...,29
247,His Girl Friday,comedy drama romance howardhawks carygrant r...,25
248,The Straight Story,biography drama davidlynch sissyspacek janega...,25


In [ ]:
data = df_final['final_words']
labels = df_final['Title']
data, labels

(0      crime  drama frankdarabont timrobbins morganfr...
 1      crime  drama francisfordcoppola marlonbrando a...
 2      crime  drama francisfordcoppola alpacino rober...
 3      action  crime  drama christophernolan christia...
 4      crime  drama sidneylumet martinbalsam johnfied...
                              ...                        
 245    drama  film-noir billywilder raymilland janewy...
 246    drama destindanielcretton brielarson johngalla...
 247    comedy  drama  romance howardhawks carygrant r...
 248    biography  drama davidlynch sissyspacek janega...
 249    drama dannyboyle loveleentandan devpatel saura...
 Name: final_words, Length: 250, dtype: object, 0      The Shawshank Redemption
 1                 The Godfather
 2        The Godfather: Part II
 3               The Dark Knight
 4                  12 Angry Men
                  ...           
 245            The Lost Weekend
 246               Short Term 12
 247             His Girl Friday
 248          The 

In [ ]:
data[0]

'crime  drama frankdarabont timrobbins morganfreeman bobgunton williamsadler two imprison men bond over a number of years, find solace and eventual redemption through act of common decency.'

In [ ]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df_final['final_words'])
count_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
cos_sim = cosine_similarity(count_matrix, count_matrix)
np.round(cos_sim,4), np.round(cos_sim,4).shape

(array([[1.    , 0.2381, 0.2054, ..., 0.0385, 0.0364, 0.1521],
        [0.2381, 1.    , 0.4313, ..., 0.1768, 0.2004, 0.2795],
        [0.2054, 0.4313, 1.    , ..., 0.183 , 0.0865, 0.3376],
        ...,
        [0.0385, 0.1768, 0.183 , ..., 1.    , 0.1134, 0.1897],
        [0.0364, 0.2004, 0.0865, ..., 0.1134, 1.    , 0.0896],
        [0.1521, 0.2795, 0.3376, ..., 0.1897, 0.0896, 1.    ]]), (250, 250))

In [ ]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
df_final.set_index('Title', inplace = True)
indices = pd.Series(df_final.index)


In [ ]:
indices[:16]

0                              The Shawshank Redemption
1                                         The Godfather
2                                The Godfather: Part II
3                                       The Dark Knight
4                                          12 Angry Men
5                                      Schindler's List
6         The Lord of the Rings: The Return of the King
7                                          Pulp Fiction
8                                            Fight Club
9     The Lord of the Rings: The Fellowship of the Ring
10                                         Forrest Gump
11       Star Wars: Episode V - The Empire Strikes Back
12                                            Inception
13                The Lord of the Rings: The Two Towers
14                      One Flew Over the Cuckoo's Nest
15                                           Goodfellas
Name: Title, dtype: object

In [ ]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cos_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df_final.index)[i])
        
    return recommended_movies

In [ ]:
recommendations("The Dark Knight")

['The Dark Knight Rises',
 "The King's Speech",
 'Goodfellas',
 'Spotlight',
 'Batman Begins',
 'The Grand Budapest Hotel',
 'Kind Hearts and Coronets',
 'The Shining',
 'The Help',
 'Gandhi']

In [ ]:
idx = indices[indices == 'The Godfather'].index[0]
idx

1

In [ ]:
score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
score_series

1      1.000000
2      0.431331
128    0.381881
15     0.380304
3      0.380304
         ...   
186    0.033408
102    0.032827
166    0.026958
77     0.000000
150    0.000000
Length: 250, dtype: float64

In [ ]:
top_10_indexes = list(score_series.iloc[1:11].index)
top_10_indexes

[2, 128, 15, 3, 115, 183, 4, 229, 226, 149]

In [ ]:
recommended_movies = []

for i in top_10_indexes:
    recommended_movies.append(list(df_final.index)[i])

In [ ]:
recommended_movies

['The Godfather: Part II',
 'Fargo',
 'Goodfellas',
 'The Dark Knight',
 'The Wolf of Wall Street',
 'Cat on a Hot Tin Roof',
 '12 Angry Men',
 "The King's Speech",
 'Rope',
 'Kind Hearts and Coronets']